In [1]:
import pandas as pd
from nummobility.core.TrajectoryDF import NumPandasTraj

pdf = pd.read_csv('./data/starkey.csv')
starkey = NumPandasTraj(data_set=pdf,
                        latitude='lat',
                        longitude='lon',
                        datetime='DateTime',
                        traj_id='Id')
starkey.head(5)

lat         lon  StarkeyTime    GMDate  \
traj_id   DateTime                                                            
880109D01 1970-01-01 00:04:17  45.238700 -118.525916    230803457  08:04:17   
          1970-01-01 00:24:02  45.241063 -118.530948    230286242  08:24:02   
          1970-01-01 00:33:08  45.249747 -118.527741    230545988  08:33:08   
          1970-01-01 00:33:51  45.238440 -118.525144    230978031  08:33:51   
          1970-01-01 00:36:53  45.254177 -118.539712    230632613  08:36:53   

                                 GMTime   LocDate   LocTime  RadNum Species  \
traj_id   DateTime                                                            
880109D01 1970-01-01 00:04:17  19950424  19950424  00:04:17     409       D   
          1970-01-01 00:24:02  19950418  19950418  00:24:02     409       D   
          1970-01-01 00:33:08  19950421  19950421  00:33:08     409       D   
          1970-01-01 00:33:51  19950426  19950426  00:33:51     409       D   
          1970-01-01 00:36:53  19950422  19950422  00:36:53     409       D   

                                 UTME     UTMN  Year  Grensunr  Grensuns  \
traj_id   DateTime                                                         
880109D01 1970-01-01 00:04:17  380220  5010606    95  12:54:00  02:53:00   
          1970-01-01 00:24:02  379854  5010865    95  13:04:00  02:45:00   
          1970-01-01 00:33:08  380122  5011840    95  12:59:00  02:49:00   
          1970-01-01 00:33:51  380293  5010559    95  12:50:00  02:56:00   
          1970-01-01 00:36:53  379191  5012351    95  12:57:00  02:50:00   

                               Obswt  
traj_id   DateTime                    
880109D01 1970-01-01 00:04:17   1.42  
          1970-01-01 00:24:02   1.43  
          1970-01-01 00:33:08   1.66  
          1970-01-01 00:33:51   1.42  
          1970-01-01 00:36:53   1.48